In [106]:
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import cv2

In [107]:
def find_objects(image, min_distance):
    """
    이미지에서 윤곽선이 들어가는 오브젝트를 찾아 박스로 표기하고, 박스간의 거리가 좁으면 하나의 박스로 합쳐줍니다.
    
    Args:
        image: PIL 이미지 객체
        min_distance: 박스간의 최소 거리
    
    Returns:
        박스로 표기된 이미지 (PIL 이미지 객체)
    """

    # 이미지를 흑백으로 변환합니다.
    grayscale_image = ImageOps.grayscale(image)
    
    # Canny 알고리즘을 사용하여 윤곽선을 추출합니다.
    edges = np.array(grayscale_image.convert('L'))
    edges = cv2.Canny(edges, 100, 200)
    
    # 윤곽선을 추출합니다.
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 박스를 표기하고 거리를 계산합니다.
    draw = ImageDraw.Draw(image)
    boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        box = (x, y, x+w, y+h)
        boxes.append(box)
    
    print(boxes)

    
    # 거리가 좁은 박스를 하나의 박스로 합칩니다.
    merged_boxes = []
    for i in range(len(boxes)):
        merged_box = boxes[i]
        
        for j in range(i+1, len(boxes)):
            if is_close_boxes(merged_box, boxes[j], min_distance) == True:
                merged_box = merge_boxes(merged_box, boxes[j])
                # print(merged_box, j)
                merged_boxes.append(merged_box)
            
    print(merged_boxes)
    
    # 합쳐진 박스를 표기합니다.
    for box in merged_boxes:
        draw.rectangle(box, outline=(33, 33, 33))
    
    return image

In [108]:
def is_close_boxes(box1, box2, min_distance):
    """
    두 박스의 거리가 좁은지 확인합니다.
    
    Args:
        box1: 첫 번째 박스
        box2: 두 번째 박스
        min_distance: 최소 거리
    
    Returns:
        True if 두 박스의 거리가 좁으면, False otherwise
    """

    b1_x, b1_y, b1_w, b1_h = box1
    
    box1_center_x = (b1_x + b1_w) / 2
    box1_center_y = (b1_y + b1_h) / 2

    
    b2_x, b2_y, b2_w, b2_h = box2

    box2_center_x = (b2_x + b2_w) / 2
    box2_center_y = (b2_y + b2_h) / 2
    
    # 두 박스의 중심점 간의 거리를 계산합니다.
    distance = np.sqrt(
        ((box1_center_x - box1_center_y)**2) +
        ((box2_center_x - box2_center_y)**2)
    )

    # print(distance, distance <= min_distance)

    return distance <= min_distance

In [109]:
def merge_boxes(box1, box2):
    """
    두 박스를 하나의 박스로 합칩니다.
    
    Args:
    box1: 첫 번째 박스
    box2: 두 번째 박스
    
    Returns:
    합쳐진 박스
    """
    
    b1_x, b1_y, b1_w, b1_h = box1
    b2_x, b2_y, b2_w, b2_h = box2
    
    return (
        min(b1_x, b2_x), # X
        min(b1_y, b2_y), # Y
        max(b1_w, b2_w), # W
        max(b1_h, b2_h), # H
    )

In [110]:
import time

# 이미지 불러오기
image = Image.open("../dataset_dir/13.한국어글자체/01.손글씨/01_handwriting_sentence_images/1_sentence/00000008.png")

# 오브젝트 찾기
object_image = find_objects(image, min_distance=5)

# 결과 이미지 저장
object_image.save(f"./image/untitle5-{time.time()}.jpg")

[(0, 174, 30, 176), (1821, 143, 1860, 157), (2825, 118, 2881, 128), (2724, 117, 2775, 156), (1805, 117, 1862, 164), (2407, 114, 2444, 159), (3116, 113, 3165, 151), (2631, 113, 2633, 114), (928, 112, 929, 114), (2015, 111, 2019, 114), (1351, 107, 1403, 149), (363, 105, 368, 108), (2010, 99, 2012, 102), (2736, 98, 2740, 100), (2012, 98, 2014, 102), (930, 98, 931, 101), (890, 98, 930, 122), (623, 98, 625, 101), (1212, 96, 1263, 146), (927, 96, 928, 97), (2721, 94, 2738, 101), (2011, 94, 2013, 96), (1782, 94, 1845, 110), (2006, 93, 2010, 98), (1485, 93, 1527, 140), (619, 92, 689, 136), (2011, 91, 2012, 93), (1583, 91, 1658, 134), (1290, 86, 1291, 87), (1285, 86, 1287, 88), (2752, 82, 2756, 86), (3471, 78, 3516, 118), (3342, 78, 3379, 103), (2470, 78, 2507, 121), (2349, 78, 2403, 131), (3534, 74, 3547, 150), (2824, 74, 2860, 111), (2631, 70, 2632, 71), (3000, 69, 3064, 135), (2083, 68, 2124, 131), (3325, 67, 3379, 93), (2879, 67, 2898, 152), (1949, 64, 1995, 143), (3380, 63, 3435, 140), (25